# Loading import library

In [1]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from rdkit.Chem.Scaffolds.MurckoScaffold import MakeScaffoldGeneric
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles


from scripts import metrics
import importlib as imp
imp.reload(metrics)

<module 'scripts.metrics' from '/home/filv/PhD/github/recall_metrics/scripts/metrics.py'>

# Funktion for calculate metrics

In [2]:
def calculate_metrics(output_set, recall_set, number, scaffold_type, type_cluster, generator, receptor, save=True):

    mt = metrics.Metrics(type_cluster, scaffold_type, generator, number, receptor, save)     
    mt.load(output_set, recall_set)
    result = mt.calculate_metrics_with_return()
    display(result)
    return result

## Count metrics 
- Need the change parameters depend on which type of scaffold/cluster/generator/target you have

In [8]:
type_cluster = 'dis' #options: 'dis'|'sim' 
numbers = [0,1,2,3,4] #options: [0]|[0,1]|[0,1,2]|[0,1,2,3]|[0,1,2,3,4] -> all combinations 0-4, depends on which calculation you have
scaffold_type = 'murcko' #options: 'csk'|'murcko'
generator = 'DrugEx_RNN' #options: 'Molpher'|'DrugEx'|'REINVENT'|'addcarbon'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'
save_options = True

In [3]:
import time
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'
save_options = True

for generator in ['DrugEx_RNN','REINVENT','GB_GA', 'addcarbon']:  #DrugEx_GT_1, DrugEx_RNN, REINVENT, GB_GA
    for scaffold_type in ['csk', 'murcko']:
        for type_cluster in ['dis', 'sim']:
            for number in [0,1,2,3,4]:
                before = time.perf_counter()
                a0 = calculate_metrics(f"data/output_sets/{receptor}/cOS_{generator}_{type_cluster}_{number}_one_column.csv", \
                                                                f"data/input_recall_sets/{receptor}/cRS_{receptor}_{type_cluster}_{number}.csv",\
                                   number, scaffold_type, type_cluster,generator,receptor,save_options)
                a0
                after = time.perf_counter()
                print('Path found after', (after - before)/60, 'minutes.')

data/output_sets/Glucocorticoid_receptor/cOS_DrugEx_RNN_dis_0_one_column.csv


[09:45:29] Explicit valence for atom # 29 K, 2, is greater than permitted
[09:45:30] Explicit valence for atom # 19 C greater than permitted
[09:45:30] Explicit valence for atom # 29 C greater than permitted
[09:45:30] Explicit valence for atom # 1 C greater than permitted
[09:45:30] Explicit valence for atom # 34 C greater than permitted
[09:45:31] Explicit valence for atom # 34 K, 2, is greater than permitted
[09:45:31] Explicit valence for atom # 14 C greater than permitted
[09:45:31] Explicit valence for atom # 25 K, 2, is greater than permitted
[09:45:31] Explicit valence for atom # 12 C greater than permitted
[09:45:31] Explicit valence for atom # 23 Na, 2, is greater than permitted
[09:45:31] Explicit valence for atom # 29 K, 2, is greater than permitted
[09:45:31] Explicit valence for atom # 5 C greater than permitted
[09:45:31] Explicit valence for atom # 27 C greater than permitted
[09:45:32] Explicit valence for atom # 26 C greater than permitted
[09:45:32] Explicit valence 

add_columns_same_like_input_function


/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

Save


...

Path found after 2.290585432533347 minutes.
data/output_sets/Glucocorticoid_receptor/cOS_addcarbon_sim_4_one_column.csv
add_columns_same_like_input_function


/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/github/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

Save


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_set,SESY,ASER,CwASo
0,addcarbon_4,sim,murcko,3270,1000000,3/40,0.075,0.229358,0.00075,0.00327,0.003239,3239


Path found after 2.1633153484833505 minutes.


## Calculate acarage value and save csv
- Do this step after calculate all metrics for each cluster

In [4]:
def avarage_value(number, type_cluster, scaffold_type, generator, receptor):

    path = f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/"
    link = {}
    for x in range(number):
        link[f'link_{x}'] = f'{path}metrics_cluster_{x}_{type_cluster}_{generator}.csv'

    df = pd.DataFrame()
    for x,y in link.items():
        a = pd.read_csv(y)
        df = pd.concat([df, a], ignore_index=True)
        
    
    a = df.mean(numeric_only=True)
    df.loc[len(df)] = [f"{generator}_mean", type_cluster, scaffold_type, a['USo'], a['SSo'], '-',\
                       a['TUPOR'],a['TUPOR_unique'],a['TUPOR_set'], a['SESY'], a['ASER'], a['CwASo']]
    df = df.round(7)
    dff = df.copy()
    dff['SSo'] = dff['SSo'].apply(lambda x : "{:,}".format(x))
    dff['USo'] = dff['USo'].apply(lambda x : "{:,}".format(x))
    dff['CwASo'] = dff['CwASo'].apply(lambda x : "{:,}".format(x))

    display(df)

    mean = pd.DataFrame()
    mean = df.loc[number].to_frame().T
                                        
    display(mean)
    dff.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean_with_coma.csv", index = False)
    df.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean.csv", index = False)
    dff.to_html(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean.html", index = False)
    mean.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}_mean_{scaffold_type}_{type_cluster}.csv", index=False) #float_format='%.7f'

In [20]:
type_cluster = 'dis' #options: 'dis'|'sim' 
number = 5 #options: 1-5
scaffold_type = 'murcko' #options: 'csk'|'murcko'
generator = 'DrugEx_RNN' #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'``

In [5]:
for generator in ['Molpher','DrugEx_GT','DrugEx_GT_1','DrugEx_RNN','REINVENT','GB_GA', 'addcarbon']:
    for scaffold_type in ['csk', 'murcko']:
        for type_cluster in ['dis', 'sim']:
            number = 5
            receptor = 'Glucocorticoid_receptor'
            avarage_value(number, type_cluster, scaffold_type, generator, receptor)

,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_set,SESY,ASER,CwASo
0,Molpher_0,dis,csk,104715.0,916546.0,15/40,0.375000,0.035812,0.004091,0.114250,0.003291,3016.0
1,Molpher_1,dis,csk,119055.0,1070030.0,15/23,0.652174,0.054779,0.006095,0.111263,0.001106,1183.0
2,Molpher_2,dis,csk,105240.0,906891.0,9/40,0.225000,0.021380,0.002481,0.116045,0.000114,103.0
3,Molpher_3,dis,csk,89696.0,693460.0,20/37,0.540540,0.060264,0.007795,0.129346,0.006726,4664.0
4,Molpher_4,dis,csk,81634.0,532962.0,34/43,0.790698,0.096859,0.014836,0.153170,0.011708,6240.0
5,Molpher_mean,dis,csk,100068.0,823977.8,-,0.516682,0.053819,0.007060,0.124815,0.004589,3041.2


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_set,SESY,ASER,CwASo
5,Molpher_mean,dis,csk,100068.0,823977.8,-,0.516682,0.053819,0.00706,0.124815,0.004589,3041.2


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_set,SESY,ASER,CwASo
0,Molpher_0,sim,csk,109136.0,855964.0,29/38,0.763158,0.069927,0.008916,0.127501,0.014377,12306.0
1,Molpher_1,sim,csk,102110.0,818233.0,28/38,0.736842,0.072162,0.009005,0.124793,0.020957,17148.0
2,Molpher_2,sim,csk,101071.0,804684.0,23/37,0.621622,0.061504,0.007725,0.125603,0.013162,10591.0
3,Molpher_3,sim,csk,98457.0,801639.0,26/35,0.742857,0.075450,0.009267,0.122820,0.012450,9980.0
4,Molpher_4,sim,csk,108264.0,901697.0,22/35,0.628571,0.058059,0.006971,0.120067,0.004182,3771.0
5,Molpher_mean,sim,csk,103807.6,836443.4,-,0.698610,0.067420,0.008377,0.124157,0.013026,10759.2


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_set,SESY,ASER,CwASo
5,Molpher_mean,sim,csk,103807.6,836443.4,-,0.69861,0.06742,0.008377,0.124157,0.013026,10759.2


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_set,SESY,ASER,CwASo
0,Molpher_0,dis,murcko,226357.0,916551.0,9/92,0.097826,0.004322,0.001067,0.246966,0.001137,1042.0
1,Molpher_1,dis,murcko,258940.0,1070030.0,8/34,0.235294,0.009087,0.002199,0.241993,0.000024,26.0
2,Molpher_2,dis,murcko,228962.0,906900.0,3/64,0.046875,0.002047,0.000517,0.252467,0.000036,33.0
3,Molpher_3,dis,murcko,182240.0,693468.0,16/76,0.210526,0.011552,0.003036,0.262795,0.005191,3600.0
4,Molpher_4,dis,murcko,152780.0,532962.0,19/98,0.193878,0.012690,0.003638,0.286662,0.001364,727.0
5,Molpher_mean,dis,murcko,209855.8,823982.2,-,0.156880,0.007940,0.002091,0.258177,0.001551,1085.6


...

,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_set,SESY,ASER,CwASo
5,addcarbon_mean,sim,csk,863.8,1000000.0,-,0.201548,2.339508,0.002015,0.000864,0.014234,14233.8


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_set,SESY,ASER,CwASo
0,addcarbon_0,dis,murcko,2334.0,1000000.0,0/92,0.000000,0.000000,0.000000,0.002334,0.000000,0.0
1,addcarbon_1,dis,murcko,3309.0,1000000.0,1/34,0.029412,0.088884,0.000294,0.003309,0.000687,687.0
2,addcarbon_2,dis,murcko,2884.0,1000000.0,0/64,0.000000,0.000000,0.000000,0.002884,0.000000,0.0
3,addcarbon_3,dis,murcko,3202.0,1000000.0,2/76,0.026316,0.082185,0.000263,0.003202,0.000092,92.0
4,addcarbon_4,dis,murcko,3116.0,1000000.0,2/98,0.020408,0.065495,0.000204,0.003116,0.002841,2841.0
5,addcarbon_mean,dis,murcko,2969.0,1000000.0,-,0.015227,0.047313,0.000152,0.002969,0.000724,724.0


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_set,SESY,ASER,CwASo
5,addcarbon_mean,dis,murcko,2969.0,1000000.0,-,0.015227,0.047313,0.000152,0.002969,0.000724,724.0


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_set,SESY,ASER,CwASo
0,addcarbon_0,sim,murcko,2583.0,1000000.0,1/115,0.008696,0.033665,0.000087,0.002583,0.000165,165.0
1,addcarbon_1,sim,murcko,2744.0,1000000.0,4/100,0.040000,0.145773,0.000400,0.002744,0.006370,6370.0
2,addcarbon_2,sim,murcko,3064.0,1000000.0,4/58,0.068965,0.225083,0.000690,0.003064,0.001924,1924.0
3,addcarbon_3,sim,murcko,3195.0,1000000.0,3/51,0.058824,0.184111,0.000588,0.003195,0.004860,4860.0
4,addcarbon_4,sim,murcko,3270.0,1000000.0,3/40,0.075000,0.229358,0.000750,0.003270,0.003239,3239.0
5,addcarbon_mean,sim,murcko,2971.2,1000000.0,-,0.050297,0.163598,0.000503,0.002971,0.003312,3311.6


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_set,SESY,ASER,CwASo
5,addcarbon_mean,sim,murcko,2971.2,1000000.0,-,0.050297,0.163598,0.000503,0.002971,0.003312,3311.6


# Connect all mean value to one pandas DataFrame

In [6]:
def connect_mean_value(reeptor, scaffold_type, type_cluster, generators_name_list):
    path = f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/"
    link = {}
    for generator in generators_name_list:
        link[f'link_{generator}'] = f'{path}{generator}_mean_{scaffold_type}_{type_cluster}.csv'

    df = pd.DataFrame()
    for x,y in link.items():
        a = pd.read_csv(y)
        df = pd.concat([df, a], ignore_index=True)

    df = df.drop(columns = ['TUPOR_'])
    display(df)
    dff = df.copy()
    dff['SSo'] = dff['SSo'].apply(lambda x : "{:,}".format(x))
    dff['USo'] = dff['USo'].apply(lambda x : "{:,}".format(x))
    dff['CwASo'] = dff['CwASo'].apply(lambda x : "{:,}".format(x))

    
    dff.to_csv(f"{path}/mean_{scaffold_type}_{type_cluster}_with_coma.csv", index = False)
    df.to_csv(f"{path}/mean_{scaffold_type}_{type_cluster}.csv", index = False)
    dff.to_html(f"{path}/mean_{scaffold_type}_{type_cluster}.html", index = False)
    return df

In [13]:
type_cluster = 'dis' #options: 'dis'|'sim' 
scaffold_type = 'murcko' #options: 'csk'|'murcko'
generators_name_list = ['Molpher', 'DrugEx_GT', 'DrugEx_GT_1','DrugEx_RNN', 'REINVENT', 'addcarbon', 'GB_GA'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

In [14]:
df = connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)

,name,type_cluster,scaffold,USo,SSo,TUPOR,TUPOR_unique,TUPOR_set,SESY,ASER,CwASo
0,Molpher_mean,dis,murcko,209855.8,823982.2,0.156880,0.007940,0.002091,0.258177,0.001551,1085.6
1,DrugEx_GT_mean,dis,murcko,532549.2,891966.0,0.387182,0.007231,0.004338,0.597031,0.002124,1890.0
2,DrugEx_GT_1_mean,dis,murcko,554504.8,908333.0,0.333769,0.006043,0.003681,0.610324,0.002054,1851.2
3,DrugEx_RNN_mean,dis,murcko,432204.4,869210.0,0.326728,0.007511,0.003764,0.496892,0.000954,827.6
4,REINVENT_mean,dis,murcko,620549.2,964674.4,0.108451,0.001758,0.001124,0.643241,0.000209,202.2
5,addcarbon_mean,dis,murcko,2969.0,1000000.0,0.015227,0.047313,0.000152,0.002969,0.000724,724.0
6,GB_GA_mean,dis,murcko,4415.6,9998.0,0.039801,0.089322,0.039808,0.441652,0.003020,30.2


In [13]:
dff = df.copy()

In [15]:
dff['TUPOR_unique'] = df['TUPOR_unique']*10000
dff

,name,type_cluster,scaffold,USo,SSo,TUPOR,TUPOR_unique,TUPOR_set,SESY,ASER,CwASo
0,Molpher_mean,dis,csk,100068.0,823977.8,0.516682,0.054,7.000000e-07,0.124815,0.004589,3041.2
1,DrugEx_GT_mean,dis,csk,242118.8,891906.4,0.695865,0.029,8.000000e-07,0.271447,0.008831,7851.4
2,DrugEx_GT_1_mean,dis,csk,254849.2,908272.6,0.700922,0.028,8.000000e-07,0.280542,0.009991,9050.0
3,DrugEx_RNN_mean,dis,csk,110595.8,868587.2,0.665922,0.059,8.000000e-07,0.127252,0.018142,16078.4
4,REINVENT_mean,dis,csk,169670.2,964580.2,0.559826,0.034,6.000000e-07,0.175880,0.013492,13023.4
5,addcarbon_mean,dis,csk,856.8,1000000.0,0.033403,0.384,0.000000e+00,0.000857,0.001827,1827.0
6,GB_GA_mean,dis,csk,2710.0,9998.0,0.197592,0.745,1.980000e-05,0.271057,0.010821,108.2
